# Dodatne naloge

Dodatne naloge naj služijo za poglobitev spretnosti programiranja, boljšemu razumevanju snovi in vsebine vaje in spoznavanju dodatnih načinov za obdelavo in analizo medicinskih slik. Opravljanje dodatnih nalog je neobvezno, vendar pa priporočljivo.

In [3]:
import os
import shutil
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import convolve

import matplotlib.pyplot as plt

LABELS = {
    1: 'caudate',
    2: 'pallidum',
    3: 'putamen',
    4: 'thalamus',    
    5: 'cortex'
}

imgs = sitk.GetArrayFromImage(sitk.ReadImage('data/t1-images.nii.gz'))    
msks = sitk.GetArrayFromImage(sitk.ReadImage('data/gm-masks.nii.gz')) 

1. Napišite funkcijo za izgradnjo referenčnega atlasa z aritmetičnim povprečenjem in nepristransko skupinsko poravnavo slik:
```python 
    def atlasConstruction( iImages, iLabelMaps, iMaxIter ):
		return oAtlasImage, oImages, oLabelMaps
```
 kjer `iImages` predstavlja seznam (`list`) 2D slik, `iLabelMaps` pa seznam (`list`) mask značk, ki pripadajo vhodnim 2D slikam. Funkcija iterativno z **aritmetičnim povprečenjem** poravnavnih slik zgradi referenčni atlas, na katerega se nato z netogo poravnavo poravnajo vse 2D slike, nato se zgradi nov referenčni atlas, itd. Postopek naj se iterativno ponovi do `iMaxIter` števila iteracij. Funkcija naj po zadnji iteraciji vrne referenčni atlas v spremenljivki `oAtlasImage` in pa seznam poravnanih 2D slik in pripadajočih značk v spremenljivkah `oImages` in `oLabelMaps`.

 Uporabite vse prečne rezine v sliki `t1-images.nii.gz` in pripadajoče maske značk `gm-masks.nii.gz` za izgradnjo referenčnega atlasa. Prikažite referenčni atlas in preverite, da so prečne rezine in maske značk v spremenljivkah `oImages` in `oLabelMaps` medsebojno poravnane.

 Pridobljene poravnane slike in maske maske značk uporabite za razgradnjo prve rezine podobno kot pri nalogah iz vaj tako, da pri zlivanju izpustite prvo rezino. Preverite delovanje postopka `fusionMajorityVoting()` in `fusionShapeBasedAveraging()`. Vrednotite uspešnost razgradnje z Diceovim koeficientom tako, da z zlivanjem pridobljeno masko značk primerjate z masko ročno določenih značk na prvi rezini.

In [30]:
### BEGIN SOLUTION
def bsplineRegistration(iFixed, iMoving, iBSplineGridSpacing, iMaxIter):
    iFixed = sitk.GetImageFromArray(iFixed.astype('float'))
    iMoving = sitk.GetImageFromArray(iMoving.astype('float'))    
    # inicializacija postopka
    R = sitk.ImageRegistrationMethod()

    # inicializacija preslikave z B-zlepki
    bTr = sitk.BSplineTransformInitializer(iFixed, [iBSplineGridSpacing]*2)
    R.SetInitialTransform(bTr, inPlace=True)

    # inicializacija mere podobnosti
    R.SetMetricAsMattesMutualInformation(64)  # SPREMENJENO od vaj
    R.SetMetricSamplingPercentage(0.10)       # SPREMENJENO od vaj
    R.SetMetricSamplingStrategy(R.RANDOM)

    # inicializacija optimizacije
    R.SetOptimizerAsGradientDescentLineSearch(learningRate=5.0,
        numberOfIterations=iMaxIter,
        convergenceMinimumValue=1e-5,
        convergenceWindowSize=5)
    R.SetOptimizerScalesFromPhysicalShift()

    # zagon poravnave
    outTx = R.Execute(iFixed, iMoving)
    return outTx

def bsplineResample(iFixed, iMoving, iTx, iInterpType=sitk.sitkLinear):
    iFixed = sitk.GetImageFromArray(iFixed)
    iMoving = sitk.GetImageFromArray(iMoving)
    # ustvarjanje izhodne slike
    S = sitk.ResampleImageFilter()
    S.SetReferenceImage(iFixed)
    S.SetInterpolator(iInterpType)
    S.SetDefaultPixelValue(0)
    S.SetTransform(iTx)
    return sitk.GetArrayFromImage(S.Execute(iMoving))

def atlasRegistration(images, masks, maxiter=5, outiter=False, verbose=True):
    reg_images = images
    reg_masks = []
    iter = 0
    while maxiter>0:
        if verbose:
            print('iteracija: {}'.format(iter))
            iter += 1
        atlas = np.mean(images.astype('float'), axis=-1)
        if outiter:
            # shranjevanje vmesne slike atlasa
            out_atlas = itk.GetImageFromArray(atlas)
            itk.WriteImage(itk.Cast(out_atlas, itk.sitkUInt16),
                           os.path.join(pth_out, 'atlas-mean-iter{}{}'.format(
                               iter,s.DEFAULT_IMAGE_FORMAT)), True)
        
        reg_images = []
        reg_masks = []
       # atlas_img = itk.GetImageFromArray(atlas)
        for i in range(images.shape[-1]):
            if verbose:
                print('\tporavnava: {}/{}'.format(i+1, images.shape[-1]))
            img = images[:,:,i]
            msk = masks[:,:,i]
            # poravnava in vzorčenje
            reg_tx = bsplineRegistration(atlas, img, 8, 50)
            reg_img = bsplineResample(atlas, img, reg_tx)
            reg_msk = bsplineResample(atlas, msk, reg_tx, itk.sitkNearestNeighbor)
            
            reg_images.append(reg_img)
            reg_masks.append(reg_msk)
        reg_images = np.dstack(reg_images)
        maxiter -= 1
    images = reg_images
    masks = np.dstack(reg_masks)
    atlas = np.mean(images.astype('float'), axis=-1)
    return atlas, images, masks


### END SOLUTION

2. Knjižnica `SimpleITK` vključuje implementacijo postopka STAPLE (STAPLE: simultaneous truth and performance level estimation) za statistično zlivanje značk:
```python
	itk.STAPLE(labelVector, confidenceWeight=1.0, foregroundValue=1.0, maximumIterations=5)
```
 Funkcija omogoča zlivanje binarnih mask, torej le mask, ki vsebujejo le značko ospredja in ozadja. Izhod funkcije je polje verjetnosti, iz katere lahko z upragovljanjem vrednosti nad 0.5 pridobimo masko značke ospredja. Zlivanja mask z večimi značkami ta implementacija ne omogoča, vendar jo lahko izvedemo tako, da vsako značko ločeno zlijemo in nato združimo v eno masko značk.

	* Preizkusite delovanje funkcije tako, da zlijete poravnane maske značk področja $l=1$ (kavdatno jedro), ki ste jih izračunali pri prejšnji nalogi. Preverite vpliv števila iteracij na rezultat postopka in določite po vašem optimalno število iteracij. Pri tem si lahko pomagate z izračunom Diceovega koeficienta.

	* Napišite funkcijo za zlivanje mask z večimi značkami na osnovi dane implementacije postopka STAPLE:
	```python
		def fusionMultilabelSTAPLE( iLabelMaps ):
			return oLabelMap
        ```	
        kjer `iLabelMaps` predstavlja seznam (`list`) mask značk, funkcija pa vrne masko zlitih značk `oLabelMap`. Pri prejšnji nalogi pridobljene poravnane rezine v `t1-images.nii.gz` in maske maske značk `gm-masks.nii.gz` uporabite za razgradnjo prve rezine tako, da pri zlivanju izpustite prvo rezino. Vrednotite uspešnost razgradnje z Diceovim koeficientom tako, da z zlivanjem pridobljeno masko značk primerjate z masko ročno določenih značk na prvi rezini. Kateri od postopkov zlivanja vrne najboljše vrednosti Diceovih koeficientov?



In [ ]:
### BEGIN SOLUTION
def fusionMultilabelStaple(masks, labels):
    # ospredje
    out_msk = np.zeros(masks.shape[:2])
    for k in labels:
        labmask = (masks == k).astype('uint32')
        labvect = [itk.GetImageFromArray(labmask[:,:,z]) for z in range(labmask.shape[-1])]
        labfusion = itk.STAPLE(labvect, 1.0, 1.0, 50)
        # labfusion = itk.STAPLE(labvect, confidenceWeight=1.0, foregroundValue=1.0, maximumIterations=5)
        labfusion = itk.GetArrayFromImage(labfusion)
        labfusion[np.isnan(labfusion)] = 0.0
        out_msk[labfusion>0.5] = k
    return out_msk

print 'Computing staple'
out_staple = cu.to_path('fusion_staple{}'.format(s.DEFAULT_IMAGE_FORMAT), pth_out)
if not os.path.exists(out_staple):
    mv = []
    msk = iu.get_array(out_mask)
    for z in range(msk.shape[-1]):
        idx = [zi for zi in range(msk.shape[-1]) if zi != z]
        mv.append(fusionStaple(msk[:,:,idx], labels))
    itk.WriteImage(itk.GetImageFromArray(np.dstack(mv).astype('uint8')), out_staple, True)
segPerformance(out_staple, out_mask, labels)


### END SOLUTION

3. Napišite funkcijo za razgradnjo na osnovi poravnave in zlivanja večih atlasov s pomočjo lokalno uteženega glasovanja:
```python
def fusionLocallyWeightedVoting( iFixed, iMovingImages, iMovingLabelMaps ):
		return oFixedLabelMap
```
 kjer `iFixed` predstavlja referenčno 2D sliko, in `iMovingImages` pa seznam (`list`) premičnih vhodnih 2D slik. Premičnim slikam pripadajo tudi vhodne maske značk `iMovingLabelMaps`, ki jih želimo zliti v novo masko značk. Funkcija naj v spremenljivki `oFixedLabelMap` vrne v prostoru referenčne slike zlite nove značke, ki predstavljajo razgradnjo slike `iFixed`.

 Za učinkovito implementacijo križno-korelacijskega koeficienta lahko razvijete števec in imenovalec funkcije v enačbi $w_n(\mathbf{x}) = CC(\mathbf{x}|\mathcal{I}_n, \mathcal{J})$ (glej prvi zvezek) ter posamezne člene izračunate s pomočjo diskretne 2D konvolucije s kvadratnim *box* jedrom (npr. `np.ones([11, 11])`). Za diskretno konvolucijo lahko uporabite funkcijo `convolve()` v Python knjižnici `scipy.ndimage`. 

	* Pri 5. nalogi  vaj ste poravnali rezine v `t1-images.nii.gz` in maske maske značk `gm-masks.nii.gz`. Uporabite jih za razgradnjo prve rezine tako, da pri zlivanju izpustite prvo rezino. Vrednotite uspešnost razgradnje z Diceovim koeficientom tako, da z zlivanjem pridobljeno masko značk primerjate z masko ročno določenih značk na prvi rezini. Kateri od postopkov zlivanja vrne najboljše vrednosti Diceovih koeficientov?

	* Določite optimalno velikost kvadratne okolice za izračun križno-korelacijskega koeficienta tako, da dobljena razgradnja dosežete maksimalno vrednost Diceovega koeficienta.


In [ ]:
### BEGIN SOLUTION
def fusionLocalWeightedMajorityVoting(target, images, masks, labels, similarity='cc', kernel_size=11, exppar=8.0):
    # izracunaj utezi
    origin = [int(kernel_size / 2)] * 2 + [0]
    if similarity == 'msd':
        weights = convolve((images-target[:,:,np.newaxis])**2.0,
                           np.ones([kernel_size]*2+[1]),
                           origin=origin
                           )**(-1.0)
    elif similarity == 'cc':
        weights = convolve(images*target[:,:,np.newaxis], np.ones([kernel_size]*2+[1]), origin=origin)
        vari = convolve(images**2.0, np.ones([kernel_size]*2+[1]), origin=origin)
        vart = convolve(target[:,:,np.newaxis]**2.0, np.ones([kernel_size]*2+[1]), origin=origin)
        weights = (weights / np.sqrt(vari*vart + 1e-7))**exppar
        # weights = (weights / np.sum(weights, axis=-1)[:,:,np.newaxis])**5.0
    labcount = []
    # ozadje
    labcount.append(np.sum((masks == 0).astype('float')*weights, axis=-1))
    # ospredje
    lab = []
    for k in labels:
        lab.append(k)
        labcount.append(np.sum((masks == k).astype('float')*weights, axis=-1))
    idx = np.argmax(labcount,axis=0)
    out_msk = np.zeros_like(idx)
    for i in range(1,len(lab)+1):
        out_msk[idx == i] = lab[i-1]
    return out_msk

### END SOLUTION

4. Razširite funkcijo za izgradnjo referenčnega atlasa z dodatnim vhodnim parametrom iMeanType`, tj. atlasConstruction(..., iMeanType)`, kjer so možne vrednosti tega parametra `'arithmetic'`, `'geometric'` ali `'harmonic'`, glede na to s kakšnim načinom povprečenja želimo zgraditi referenčni atlas. 

    * Izgradite referenčni atlas z vsakim od treh načinov povprečenja in primerjajte dobljene slike referenčnih atlasov. Uporabite vse prečne rezine v sliki `t1-images.nii.gz` in pripadajoče maske značk `gm-masks.nii.gz` za izgradnjo referenčnega atlasa. Izvedite tri iteracije za izgradnjo atlasa (`iMaxIter=3`) in shranite povprečne slike v vsaki iteraciji ter slike prikažite. 

    * Preverite kako izbira načina povprečenja vpliva na razgradnjo slik s poravnavo in zlivanjem značk tako, da podobno kot pri prvi dodatni nalogi izvedete zlivanje ter določite Diceove koeficiente za vsako od petih $GM$ struktur ($l=\{ 1, 2, 3, 4, 5 \}$). Kateri način povprečenja in kateri način zlivanja slik da najboljši rezultat?

In [ ]:
### BEGIN SOLUTION
def atlasConstruction(images, type='mean'):
    if type == 'mean':
        return np.mean(images.astype('float'), axis=-1)
    elif type == 'geometric':
        return np.prod(images.astype('float'), axis=-1)**float(1.0/images.shape[-1])
    elif type == 'harmonic':
        return np.mean((images.astype('float') + 1e-7)**(-1.0), axis=-1)**(-1.0)

def atlasRegistration(images, masks, maxiter=5, outiter=True, verbose=True):
    reg_images = images
    reg_masks = []
    iter = 0
    while maxiter>0:
        if verbose:
            print 'iteracija: {}'.format(iter)
            iter += 1
        atlas = atlasConstruction(reg_images)
        if outiter:
            # shranjevanje vmesne slike atlasa
            out_atlas = itk.GetImageFromArray(atlas)
            itk.WriteImage(itk.Cast(out_atlas, itk.sitkUInt16),
                           os.path.join(pth_out, 'atlas-mean-iter{}{}'.format(
                               iter,s.DEFAULT_IMAGE_FORMAT)), True)
            itk.WriteImage(itk.Cast(itk.GetImageFromArray(
                atlasConstruction(reg_images,'geometric')), itk.sitkUInt16),
                           os.path.join(pth_out, 'atlas-geom-iter{}{}'.format(
                               iter, s.DEFAULT_IMAGE_FORMAT)), True)
            itk.WriteImage(itk.Cast(itk.GetImageFromArray(
                atlasConstruction(reg_images,'harmonic')), itk.sitkUInt16),
                           os.path.join(pth_out, 'atlas-harm-iter{}{}'.format(
                               iter, s.DEFAULT_IMAGE_FORMAT)), True)
        reg_images = []
        reg_masks = []
        atlas_img = itk.GetImageFromArray(atlas)
        for i in range(images.shape[-1]):
            if verbose:
                print '\tporavnava: {}/{}'.format(i+1, images.shape[-1])
            img = itk.GetImageFromArray(images[:,:,i].astype('float'))
            msk = itk.GetImageFromArray(masks[:,:,i])
            reg_tx = nonrigidRegistration(atlas_img, img)
            reg_img = nonrigidResample(atlas_img, img, reg_tx)
            reg_msk = nonrigidResample(atlas_img, msk, reg_tx, itk.sitkNearestNeighbor)
            reg_images.append(itk.GetArrayFromImage(reg_img))
            reg_masks.append(itk.GetArrayFromImage(reg_msk))
        reg_images = np.dstack(reg_images)
        maxiter -= 1
    images = reg_images
    masks = np.dstack(reg_masks)
    atlas = atlasConstruction(images)
    return atlas, images, masks
### END SOLUTION

5. Na osnovi optimalno poravnanih slik iz prejšnje naloge izgradite statističen atlas za vsako od petih $GM$ struktur ($l=\{ 1, 2, 3, 4, 5 \}$) tako, da uporabite vse maske značk `oLabelMaps`. Statističen atlas bo podan s petimi slikami z vrednostmi od 0 do 1, od katerih vsaka predstavlja prostorsko verjetnost posamezne $GM$ strukture.
